In [80]:
# create A database of embeddings
# choose an embedding model from hugging-face and use it to store emebddings

# retrieve similar vectors using the emebedding
# variance of distances and algorithmes to use for that

In [ ]:
from transformers import AutoTokenizer
from dearpygui import dearpygui as dpg 


model_path ='a_certain_path'
tokenizer = AutoTokenizer.from_pretrained()



# list collections 
`client.list_collections()`

In [5]:
# pip install chromadb==3.
import chromadb 
from chromadb.config import Settings 

In [6]:
# get the chroma client 
chroma_client = chromadb.Client()

In [7]:
# a collection 
collection = chroma_client.create_collection(name="myCollection")


In [15]:

# this will make the embeddings for us
# switch add to upset, to avoid adding the same documents
collection.add(
    documents= [
        "this is  a document",
        "this is another document"
    ], 
    metadatas=[
        {"source":"my_source",
         "page":1}, 
        {"source": "another_source"}, 
    ], 
    ids=["id1", "id2"]
)
# running this the first time will output this :
# C:\Users\monms\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz:  28%|██▊       | 22.3M/79.3M [00:49<01:56, 515kiB/s]
# that's because it will download an embedding model, (all-MiniLM-L6-V2) since this code will store the embeddings automatically of these documents

Insert of existing embedding ID: id1
Insert of existing embedding ID: id2
Add of existing embedding ID: id1
Add of existing embedding ID: id2


In [21]:
# if we already generated embeddings we could do : 
ebd1 = [1 for _ in range(384)]
ebd2 = [1 for _ in range(384)]
collection.add(
    embeddings=[ebd1, ebd2],
    documents=["This is a document", "This is another document"],
    metadatas=[{"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2"]
)

Insert of existing embedding ID: id1
Insert of existing embedding ID: id2
Add of existing embedding ID: id1
Add of existing embedding ID: id2


In [34]:
# querying the collection 
results = collection.query(
    query_texts = ["this is a document"], 
    n_results =2 ,
    include = ['distances', 'metadatas', 'embeddings', 'documents']
)


documents = results['documents'][0]
distances = results['distances'][0]

p = f""" 
documents : {documents}
distances : {distances}
"""
print(p)

 
documents : ['this is  a document', 'this is another document']
distances : [0.0, 0.3858669102191925]



# persistant chroma client 

In [99]:
from tqdm import tqdm 
from time import sleep

for _ in tqdm(range(10)): 
    sleep(1)
    pass

100%|██████████| 10/10 [00:10<00:00,  1.01s/it]


In [33]:
client = chromadb.PersistentClient(path='./data/')

In [49]:
client.heartbeat() # returns a nanosecond heartbeat. Useful for making sure the client remains connected.
# client.reset() # Empties and completely resets the database. ⚠️ This is destructive and not reversible.

1715349038978443400

# client server mode 

In [ ]:
# you gotta run it first like this 
# chroma run --path /db_path



import chromadb
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

# Collections
collecions has a name and optional embedding_function , you must specify the embedding function every time you get the collection

In [25]:
! pip install sentence-transformers

   -------------------------------------- 171.5/171.5 kB 368.1 kB/s eta 0:00:00


In [45]:
chroma_client.delete_collection("hmycollection")

In [66]:
from chromadb import Documents, EmbeddingFunction, Embeddings

from chromadb.utils import embedding_functions
default_ef = embedding_functions.DefaultEmbeddingFunction()

chroma_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# NOTE: this one doesnt work! 
# class MyEmbeddingFunction(EmbeddingFunction):
#     def __call__(self, input: Documents) -> Embeddings:
#         # embed the documents somehow
#         return [float(1) for _ in range(512)]


collection = chroma_client.create_collection(name="hmycollection", embedding_function=default_ef)

In [63]:

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)


ef = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
print('type of embedding_function : ', type(ef))


type of embedding_function :  <class 'langchain_community.embeddings.huggingface.HuggingFaceEmbeddings'>


In [67]:
chroma_client.delete_collection("mycollection")

In [69]:
# creating a collection 
collection = chroma_client.create_collection(name="mycollection", embedding_function=chroma_ef)

In [70]:
# you must supply the embedding function every time you get the collection 
collection  = chroma_client.get_collection(name="mycollection", embedding_function=chroma_ef)

In [29]:
!pip install langchain_chroma

  Using cached pulsar_client-3.5.0-cp310-cp310-win_amd64.whl.metadata (1.0 kB)
   ---------------------------------------- 525.5/525.5 kB 1.1 MB/s eta 0:00:00
Using cached pulsar_client-3.5.0-cp310-cp310-win_amd64.whl (3.3 MB)
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.5.0
    Uninstalling chromadb-0.5.0:
      Successfully uninstalled chromadb-0.5.0


In [30]:
from langchain_chroma import Chroma

db = Chroma.from_documents()

### delete a collection

In [66]:
client.delete_collection(name='my_collection')

In [73]:
collection = client.get_or_create_collection(name="my_collection")

peek = collection.peek() # returns a list of the first 10 items in the collection
count = collection.count() # returns the number of items in the collection
# res = collection.modify(name="new_name") # Rename the collection


print('peek ', peek)
print('count', count)
# print('res', res)

peek  {'ids': [], 'embeddings': [], 'metadatas': [], 'documents': [], 'uris': None, 'data': None}
count 0


### change the distance function 
we have :   
* `l2`: $d = \sum\left(A_i-B_i\right)^2$  
* `ip`: $d = 1.0 - \sum\left(A_i \times B_i\right) $  
* `cosine` $d = 1.0 - \frac{\sum\left(A_i \times B_i\right)}{\sqrt{\sum\left(A_i^2\right)} \cdot \sqrt{\sum\left(B_i^2\right)}}$  
just change the hnsw:space in metadata

In [ ]:
collection = client.create_collection(
    name="collection_name",
    metadata={"hnsw:space": "cosine"} # l2 is the default ()
)
# HNSW (Hierarchical Navigable Small World) indexing
# does this mean that it is by defeault an embedding database and here we are specifying which similarity we will give

# add documents 

In [76]:
collection.add(
    documents=["doc1", "doc2", "doc3"],
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2]],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}],
    ids=["id1", "id2", "id3"]
)


In [101]:
res = collection.peek()
res

{'ids': ['id1', 'id2', 'id3'],
 'embeddings': [[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2]],
 'metadatas': [{'chapter': '3', 'verse': '16'},
  {'chapter': '3', 'verse': '5'},
  {'chapter': '29', 'verse': '11'}],
 'documents': ['doc1', 'doc2', 'doc3'],
 'uris': None,
 'data': None}

# you can also store embeddings and meta data only and documents elsewhere

In [ ]:
# collection.add(
#     embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2], ...],
#     metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
#     ids=["id1", "id2", "id3", ...]
# )
